# Introduction

First, we will prepare the data into a format that is suitable for training our models. For this, we 
- discard features with no value for prediction,
- go over the datatypes, 
- check missing/'bad' values, 
- make sure there are no instances with values that don't make sense logically, 
- encode the data into machine-readable format.

In [41]:
# Import required packages

import pandas as pd

<span style="color: lightgreen; font-weight: bold;"> 1. Loading in the dataset and making a copy for preparing the data </span>

In [ ]:
df = pd.read_csv('00_SBAnational.csv').copy()
df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_27128\2101721642.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('0-SBAnational.csv').copy()


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


Explanation for each column:
| Column | Meaning |
| ------ | ----- |
| LoanNr_ChkDgt | Loan ID |
| Name | Borrower name |
| City | Borrower city |
| State | Borrower state |
| Zip | Borrower zip code |
| Bank | Bank name |
| BankState | Bank state |
| NAICS | North American industry classification system code (See table below for code meanings) |
| ApprovalDate | Date when loan was approved |
| ApprovalFY | Fiscal year of approval |
| Term | Loan term in months |
| NoEmp | Number of business employees |
| NewExist | 1 = Existing business, 2 = New business |
| CreateJob | Projected number of jobs created as a result of receiving the loan |
| RetainedJob | Projected number of jobs retained as a result of receiving the loan |
| FranchiseCode | Franchise code, (00000 or 00001) = No franchise |
| UrbanRural | 1 = Urban, 2 = rural, 0 = undefined |
| RevLineCr | Revolving line of credit: Y = Yes, N = No |
| LowDoc | LowDoc Loan Program: Y = Yes, N = No |
| ChgOffDate | The date when a loan is declared to be in default |
| DisbursementDate | Disbursement date |
| DisbursementGross | Amount disbursed |
| BalanceGross | Gross amount outstanding |
| MIS_Status | Final performance status of the loan, charged off = CHGOFF, Paid in full = PIF |
| ChgOffPrinGr | Charged-off amount |
| GrAppv | Gross amount of loan approved by bank |
| SBA_Appv | SBA’s guaranteed amount of approved loan (what the SBA will reimburse the lender in case of charge off) |

Explanation for the first two digits of NAICS code:
| Digits | Industry |
| ------ | ----- |
| 11 | Agriculture, forestry, fishing and hunting |
| 21 | Mining, quarrying, and oil and gas extraction |
| 22 | Utilities |
| 23 | Construction |
| 31–33 | Manufacturing |
| 42 | Wholesale trade |
| 44–45 | Retail trade |
| 48–49 | Transportation and warehousing |
| 51 | Information |
| 52 | Finance and insurance |
| 53 | Real estate and rental and leasing |
| 54 | Professional, scientific, and technical services |
| 55 | Management of companies and enterprises |
| 56 | Administrative and support and waste management and remediation services |
| 61 | Educational services |
| 62 | Health care and social assistance |
| 71 | Arts, entertainment, and recreation |
| 72 | Accommodation and food services |
| 81 | Other services (except public administration) |
| 92 | Public administration |

<span style="color: lightgreen; font-weight: bold;"> 2. Discarding attributes that we do not need </span>

This dataset has a lot of attributes (27), most of them are not needed in predicting the loan amount a company will be approved for (GrAppv). Let's get rid of those columns.

```LoanNr_ChkDgt```; ```Name```; ```City```; ```Zip```; ```Bank```; ```BankState``` -> These attributes do not have predictive value.

```ApprovalDate``` -> We can drop this column because ```ApprovalFY``` is better to use later for analyzing loan volume and size change over the years.

```ChgOffDate```; ```DisbursementDate```; ```DisbursementGross```; ```BalanceGross```; ```MIS_Status```; ```ChgOffPrinGr```; ```SBA_Appv``` -> These attributes also do not have predictive value, in addition to that they happen post loan approval.

In [43]:
columnsToDrop = ['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'ChgOffDate', 'DisbursementDate', 
                 'DisbursementGross', 'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'SBA_Appv']

df.drop(columns=columnsToDrop, inplace=True)
print(df.columns)

Index(['State', 'NAICS', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'GrAppv'],
      dtype='object')


Now we are left with

**Feature attributes (12)**: ```State```, ```NAICS```, ```ApprovalFY```, ```Term```, ```NoEmp```, ```NewExist```, ```CreateJob```, ```RetainedJob```, ```FranchiseCode```, ```UrbanRural```, ```RevLineCr```, ```LowDoc```

**Target (1)**: ```GrAppv```

<span style="color: lightgreen; font-weight: bold;"> 3. Checking datatypes & cleaning values </span>

In [44]:
df.dtypes

State             object
NAICS              int64
ApprovalFY        object
Term               int64
NoEmp              int64
NewExist         float64
CreateJob          int64
RetainedJob        int64
FranchiseCode      int64
UrbanRural         int64
RevLineCr         object
LowDoc            object
GrAppv            object
dtype: object

We have some conflicts with datatypes, let's fix them:

- ```ApprovalFY``` is object but should be numeric (year)
- ```GrAppv``` is the numeric target we are predicting, currently the datatype is object
- ```NewExist``` is float64, it is better to have it as int64

In [45]:
df['ApprovalFY'] = pd.to_numeric(df['ApprovalFY'], errors='coerce')
df['ApprovalFY'] = df['ApprovalFY'].astype('Int64')

df['GrAppv'] = (df['GrAppv']
                .astype(str)
                .str.replace('[^0-9.-]', '', regex=True)) # Remove the '$' characters in front of the loan amounts and any other characters that are not numerical
df['GrAppv'] = pd.to_numeric(df['GrAppv'], errors='coerce')
df['GrAppv'] = df['GrAppv'].astype('Int64')

df['NewExist'] = df['NewExist'].astype('Int64')

Next, we deal with missing values.

In [46]:
df.isna().sum()

State              14
NAICS               0
ApprovalFY         18
Term                0
NoEmp               0
NewExist          136
CreateJob           0
RetainedJob         0
FranchiseCode       0
UrbanRural          0
RevLineCr        4528
LowDoc           2582
GrAppv              0
dtype: int64

The amount of missing values is very small compared to the size of our dataset (899164 rows). We will discard the rows entirely rather than imputing.

In [47]:
df.dropna(axis=0, how='any', inplace=True)

Let's check if there are any 'bad' values

In [48]:
badValues = ['n/a', 'na', 'nan', 'null', 'unknown', '']

badCounts = {}

for col in df.columns:
    s = df[col].astype(str).str.lower()
    badCounts[col] = s.isin(badValues).sum()

for key, value in badCounts.items():
    print(f"{key}: {value} bad values")

State: 0 bad values
NAICS: 0 bad values
ApprovalFY: 0 bad values
Term: 0 bad values
NoEmp: 0 bad values
NewExist: 0 bad values
CreateJob: 0 bad values
RetainedJob: 0 bad values
FranchiseCode: 0 bad values
UrbanRural: 0 bad values
RevLineCr: 0 bad values
LowDoc: 0 bad values
GrAppv: 0 bad values


We have dealt with missing values. Next, let's check each column separately

`State`

In [49]:
# Value should be a string with length 2
maskState = df['State'].astype(str).str.len() == 2
print((~maskState).sum()) # amount of instances that do not match

0


`NAICS`

In [50]:
# Value should be an integer >=11
maskNAICS = df['NAICS'] >= 11
print((~maskNAICS).sum())

199734


We have 199734 instances that do not have a NAICS code. Since NAICS is important in our goal, we will discard these rows entirely. After discarding, we still have ~700k instances, which is plenty.

In [51]:
df = df[maskNAICS]

# Also, we do not need the full NAICS code. We only need the first 2 digits.
df['NAICS'] = (df['NAICS'] // 10000).astype('Int64')

Additionally, let's check if the NAICS code is defined (see table in the first step).

In [52]:
definedCodes = [11, 21, 22, 23, 31, 32, 33, 42, 44, 45, 48, 49, 51, 52, 53, 54, 55, 56, 61, 62, 71, 72, 81, 92]

maskNAICS_02 = df['NAICS'].isin(definedCodes)
print((~maskNAICS_02).sum())

0


`ApprovalFY`

In [53]:
# Value should be an integer (year)
maskAFY = (df['ApprovalFY'] > 1900) & (df['ApprovalFY'] < 2026)
print((~maskAFY).sum())

0


`Term`

In [54]:
# Value should be number of months
maskTerm = df['Term'] > 0
print((~maskTerm).sum())

730


730 instances have a term that is 0 or smaller. Let's get rid of those rows.

In [55]:
df = df[maskTerm]

`NoEmp`

In [56]:
# Value should be an integer >= 0 (number of employees)
maskNE = df['NoEmp'] >= 0
print((~maskNE).sum())

0


`NewExist`

In [57]:
# Value should be an integer, either 1 or 2 (1 = existing business, 2 = new business)
maskNewExist = (df['NewExist'] == 1) | (df['NewExist'] == 2)
print((~maskNewExist).sum())

729


In [58]:
df = df[maskNewExist]

`CreateJob`

In [59]:
# Value should be an integer >= 0
maskCJ = df['CreateJob'] >= 0
print((~maskCJ).sum())

0


`RetainedJob`

In [60]:
# Value should be an integer >= 0, smaller or equal to NoEmp + CreateJob (because the number of retained jobs can't be bigger than total jobs)
maskRJ = (df['RetainedJob'] >= 0) & (df['RetainedJob'] <= df['NoEmp'] + df['CreateJob'])
print((~maskRJ).sum())

2576


In [61]:
df = df[maskRJ]

`FranchiseCode`

In [62]:
# Value should be an integer >= 0
maskFC = df['FranchiseCode'] >= 0
print((~maskFC).sum())

0


`UrbanRural`

In [63]:
# Value should be an integer: 1, 2  (1 = Urban, 2 = rural, 0 = undefined)
maskUR = df['UrbanRural'].isin([1,2])
print((~maskUR).sum())

141932


In [64]:
df = df[maskUR]

`RevLineCr`

In [65]:
# Value should be either Y (yes) or N (no)
maskRLC = df['RevLineCr'].isin(['Y','N'])
print((~maskRLC).sum())

195447


In [66]:
df = df[maskRLC]

`LowDoc`

In [67]:
# Value should be either Y (yes) or N (no)
maskLD = df['LowDoc'].isin(['Y','N'])
print((~maskLD).sum())

1100


In [68]:
df = df[maskLD]

`GrAppv`

In [69]:
# Value should be integer > 0
maskGA = df['GrAppv'] > 0
print((~maskGA).sum())

0


<span style="color: lightgreen; font-weight: bold;"> 4. Encoding categorical variables </span>

The next step is encoding these values to a machine-readable format. 


**Binary categories** with values like `1/2`, `Y/N` -> we map to `0/1`

**Multi-class categoricals** with many different values -> we use one-hot encoding or leave as numeric


In [70]:
for col in df:
    print(f"Column '{col}' has {df[col].nunique()} unique values.")

Column 'State' has 51 unique values.
Column 'NAICS' has 24 unique values.
Column 'ApprovalFY' has 21 unique values.
Column 'Term' has 327 unique values.
Column 'NoEmp' has 332 unique values.
Column 'NewExist' has 2 unique values.
Column 'CreateJob' has 177 unique values.
Column 'RetainedJob' has 254 unique values.
Column 'FranchiseCode' has 1411 unique values.
Column 'UrbanRural' has 2 unique values.
Column 'RevLineCr' has 2 unique values.
Column 'LowDoc' has 2 unique values.
Column 'GrAppv' has 8978 unique values.


In [71]:
# State - we use one-hot encoding
df = pd.get_dummies(df, columns=['State'], drop_first=True)

In [72]:
# NAICS - we use one-hot encoding
df = pd.get_dummies(df, columns=['NAICS'], prefix='NAICS', drop_first=True)

In [73]:
# ApprovalFY - we split years into ranges like 1950-1960, 1960-1970 etc. This turns the year into an ordinal feature.

bins = [1980, 1990, 2000, 2010, 2020, 2030]
labels = ['1980s', '1990s', '2000s', '2010s', '2020s']
df['ApprovalFY_bin'] = pd.cut(df['ApprovalFY'], bins=bins, labels=labels, right=False)
print(df[['ApprovalFY', 'ApprovalFY_bin']].head())
print("-----------------------------------")

# Now we use one-hot encoding
df = pd.get_dummies(df, columns=['ApprovalFY_bin'], prefix='AFY', dummy_na=False)
print([c for c in df.columns if c.startswith('AFY_')])

# Discard ApprovalFY
df = df.drop(columns=['ApprovalFY'])

    ApprovalFY ApprovalFY_bin
12        2006          2000s
24        2006          2000s
28        2006          2000s
30        2006          2000s
32        2006          2000s
-----------------------------------
['AFY_1980s', 'AFY_1990s', 'AFY_2000s', 'AFY_2010s', 'AFY_2020s']


`Term` and `NoEmp` - we leave as numeric.

In [74]:
# NewExist - binary categorical: map to 0/1
# We create a new column 'NewBusiness', value 1 means the business is new.
df['NewBusiness'] = df['NewExist'].map({1: 0, 2: 1})

df = df.drop(columns=['NewExist'])

`CreateJob` and `RetainedJob` - we leave as numeric.

In [75]:
# FranchiseCode - we will lose franchise codes and replace this by mapping them by if they are a franchise or not.
df['IsFranchise'] = (df['FranchiseCode'] > 1).astype(int)
df = df.drop(columns=['FranchiseCode'])

In [76]:
# UrbanRural - map to two binary features
df['Urban'] = (df['UrbanRural'] == 1).astype(int)
df['Rural'] = (df['UrbanRural'] == 2).astype(int)
df = df.drop(columns=['UrbanRural'])

In [77]:
# RevLineCr - binary categorical: map to 0/1
df['RevLineCr'] = df['RevLineCr'].map({'N': 0, 'Y': 1})

In [78]:
# LowDoc - binary categorical: map to 0/1
df['LowDoc'] = df['LowDoc'].map({'N': 0, 'Y': 1})

`GrAppv` - this is our target, no encoding.

<span style="color: lightgreen; font-weight: bold;"> 5. Saving the prepared data </span>

Now the data is prepared and ready for modelling.

In [79]:
print(f"Instances: {len(df)}\nFeatures: {len(df.columns)}")

Instances: 349641
Features: 89


After preparing, the number of instances and features has changed noticeably.

Let's save the prepared data to a new `.csv` file.

In [ ]:
df.to_csv("02_Prepared_data.csv", index=False)

⚠️ Scaling has not been applied in this preparation, it will be done seperately in the linear regression model.